In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
from warnings import warn
import dask.dataframe as dd
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', '{:.2f}'.format)

global df_title_basics, df_title_ratings, df_tmdb # crear variables globales
pd.set_option('display.float_format', lambda x: '%0.3f' % x)

In [4]:
# Abrir los dataset lismpios. 
df_title_basics = pd.read_csv(r'C:\Users\Chamo\Documents\title_basics_procesado1.csv',sep='\t')
df_title_ratings = pd.read_csv(r'C:\Users\Chamo\Documents\df_title_ratings.csv', sep='\t')
df_tmdb = pd.read_csv(r'C:\Users\Chamo\Documents\df_tmdb.csv', sep=',')

In [5]:
 
 # combertir la columna de años a tipo de datos de tiempo (datatime)
df_title_basics['startYear'] = pd.to_datetime(df_title_basics['startYear'], format='%Y-%m-%d', errors='coerce')

 # incluir solo los años relevantes (1990-2024)
df_title_basics = df_title_basics[df_title_basics['startYear'].between('1900-01-01', '2024-01-01') & (df_title_basics['runtimeMinutes'].between(15, 240))]
 
 # filtral los rating relevantes (500+), los de menos de 500 votos no son relevantes.
df_title_ratings = df_title_ratings[(df_title_ratings['numVotes']>500) & (df_title_ratings['averageRating']>0)]
 
 # incluir solo las peliculas con un budget mayor o igual a 500k de los anos 1990-2024.
df_tmdb = df_tmdb[df_tmdb['release_date'].between('1900-01-01', '2024-01-01')]
df_tmdb = df_tmdb[df_tmdb['budget']>=500000]

------------------------------------------------------------------------------------------------------------------------------------------

Pregunta 1: ¿Cual es la evolución del rating por tipo de contenido?

In [6]:
def pregunta_1():
    global df_title_basics, df_title_ratings, df_tmdb
    
    # Grafico de pizza.
    df_title_basics_pizza = df_title_basics['titleType'].value_counts().reset_index()
    df_title_basics_pizza.columns = ['Tipo de Contenido', 'Conteo']
    
    
    # Crea el grafico de Pizza
    fig_pie = px.pie(df_title_basics_pizza, names='Tipo de Contenido', values='Conteo',
                 title='Distribución por Tipo de Contenido (Movies vs TVSeries)')
             
    # Mejorar la interactivida y apariencia del grafico de pizza
    fig_pie.update_traces(textposition='inside', textinfo='percent+label')
    fig_pie.update_layout(showlegend=True, template='plotly_dark')
    # Fin grafico de Pizza
    
    # Grafico de linea tratamiento.
    
    df_merged = pd.merge(df_title_basics, df_title_ratings, on='tconst') # combinar DF 
    
    df_merged['year'] = df_merged['startYear'].dt.year  #crear columna donde solo contenga los años 
    
    # obtengo los años unico para luego poder hacer el intervalos de los años
    years_movies = df_merged[df_merged['titleType']=='movie']['year'].unique()
    years_tvSeries = df_merged[df_merged['titleType']=='tvSeries']['year'].unique()
    
    # intervalos para encontral en rango comun de años. Rango comun = 1947-2024
    min_year = max(min(years_movies), min(years_tvSeries))
    max_year = min(max(years_movies), max(years_tvSeries))
    
    # selecciono solo los anos que estan en el rango en comun. comenso en 1990
    df_merged_filtered = df_merged[df_merged['year'].between(1990, max_year)]
    
    # Agrupar por las columnas year y title type y ontener la media arimetica de la columna average Rating 
    evolucion_rating = df_merged_filtered.groupby(['year', 'titleType'])['averageRating'].mean().unstack()
    
    
    # Convertir el DataFrame agrupado a formato largo para usar con Plotly
    evolucion_rating_reset = evolucion_rating.reset_index()
    evolucion_rating_melted = evolucion_rating_reset.melt(id_vars=['year'], value_vars=['movie', 'tvSeries'],
                                                      var_name='Tipo de Contenido', value_name='Rating Promedio')
    
    
    # Reset a 3 decimales.
    evolucion_rating_melted['Rating Promedio'] = evolucion_rating_melted['Rating Promedio'].round(3)
    
    # Crear el gráfico de linea.
    fig_line = px.line(evolucion_rating_melted, x='year', y='Rating Promedio', color='Tipo de Contenido',
             markers=True, title='Distribución por Tipo de Contenido (Movies vs TVSeries)')

    # Mejorar la interactividad y apariencia
    fig_line.update_layout(hovermode='x unified', template='plotly_dark')
    fig_line.update_traces(marker=dict(size=5), line=dict(width=2))
     # Fin grafico de linea




    #Combinar los 2 graficos (Pizza y Linea)
    # Crear el subplot
    fig = make_subplots(
        
        rows=1, cols=2,
        specs=[[{'type': 'xy'}, {'type': 'domain'}]],
        column_widths=[0.60, 0.2],  # Ajustar el ancho de las columnas
        subplot_titles=("Rating promedio por año y contenido", "Distribución por Tipo de Contenido"),
        horizontal_spacing=0.03 ) # Eliminar el espacio horizontal entre las subtramas


    # Añadir el gráfico de líneas al subplot
    for trace in fig_line.data:
        fig.add_trace(trace, row=1, col=1)

    # Añadir el gráfico de pizza al subplot usando las trazas del gráfico de pizza existente
        for trace in fig_pie.data:
            fig.add_trace(trace, row=1, col=2)
    

    # Ajustar la posición y el tamaño del gráfico de pizza
    fig.update_layout(
        height=300,
        width=1000,
        showlegend=True,
        template='plotly_dark',
        margin=dict(l=20, r=20, t=50, b=20),  # Ajustar los márgenes
        legend=dict(x=1, y=1)  # Posicionar la leyenda más cerca del gráfico de pizza
)

    # Mostrar el gráfico
    fig.show()
    

pregunta_1()
    

El análisis revela que las películas dominan el mercado del entretenimiento con un 82.5% del contenido, frente al 17.5% de las
series de TV. Sin embargo, las series han mantenido consistentemente un rating superior, por encima de 7 puntos, mientras que las
películas han oscilado alrededor de los 6 puntos, lo que sugiere una mayor satisfacción del público con este formato

Notablemente, a partir de 2020, se observa un punto de inflexión significativo: el rating de las películas muestra un ascenso
pronunciado, acercándose a los niveles de las series. Esta tendencia reciente sugiere un cambio en las preferencias de la audiencia en el  
mercado del entretenimiento.

---------------------------------------------------------------------------------------------------------------------------------------

Pregunta numero 2: ¿Cual es el rating promedio por género?

In [8]:
def pregunta_2():
    global df_title_basics, df_title_ratings, df_tmdb
    
    
    # separar los generos en valores individuales (uso assign y split para mejor comprencion)
    df_exploded = df_title_basics.assign(genre=df_title_basics['genres'].str.split(',').explode('genre'))
    # df_exploded dataset con los generos separados.
    
     # uniendo los dataset por la columna tconst 
    df_merged_pg2 = df_exploded.merge(df_title_ratings, on='tconst') 
    
    # lista de generos no deseados.
    unwanted_genres = ['Short', 'Adult', 'News', 'Game-Show', 'Adult', 'Talk-Show', 'Western', 'Film-Noir','Thriller', 'TV Movie']

    # filtral los generos no deseados.
    df_merged_pg2 = df_merged_pg2[~df_merged_pg2['genre'].isin(unwanted_genres)]
    
    '''"Muestreo de 1000 datos para obtener 
        medias con higualdad de provavilidades 
    ''' 
    # Creando DF vacio para llenar con los datos del muestreo.
    df_genre_rating_sample = pd.DataFrame()
    
    # Agrupo por genero
    df_genre_ratings = df_merged_pg2.groupby('genre')
    """
     verifico mi generos para ver si nesecito muestreo con reemplazo. 
     print(df_genre_ratings['genre'].value_counts())
     
     y utiliso if y else, para usar reemplaso si el genero tiene menos de 1000 valores
     y sin reemplaso si tiene mas de 1000 valores.
     
     df_genre_rating_sample['genre'].value_counts().head(20) # reviso si esta correcto 
     con 1000 muestras cada genero.
    """
    for genre, group in df_genre_ratings:
        if len(group)< 1000:
            genre_sample = group.sample(n=1000, replace=True, random_state=100)
        else:
            genre_sample = group.sample(n=1000, random_state=100)
    
        df_genre_rating_sample = pd.concat([df_genre_rating_sample, genre_sample], ignore_index=True) 
        
         # Agrupar por genero y obtener la media arimetica de los rating por genero.
        df_genre_ratings = df_genre_rating_sample.groupby('genre')['averageRating'].mean().reset_index()
        df_genre_ratings.columns = ['Genero', 'Rating Promedio']
        
        # creo un df con el total de cada genero.
        df_genre_count = df_exploded['genre'].value_counts().reset_index()
        df_genre_count.columns = ['Genero', 'Count']
        
        # Uniendo el df que contiene la contida de por generos y el que contiene el rating promedo por genero.
        df_combined = df_genre_ratings.merge(df_genre_count, on='Genero')
        df_combined = df_combined.sort_values('Count', ascending=False) # organizo decendente por la columna Count.
        
        
        
        
         
          # crear la fig combinada
    fig_comb = make_subplots(specs=[[{'secondary_y': True}]])

  
  
        # crear el grafico de barras.
    fig_comb.add_trace(
        go.Bar(x=df_combined['Genero'], y=df_combined['Count'], text=df_combined['Count'],
            name='Cantidad por Genero'))

       # Anadir la linea del Rating Promedio.
    fig_comb.add_trace(
        go.Scatter(x=df_combined['Genero'], y=df_combined['Rating Promedio'],
            mode='lines+markers', name='Rating Promedio', marker=dict(color='#229954')),
            secondary_y=True)


        # Actualizar el layout
    fig_comb.update_layout(
        width=1000,
        title_text='Distribución y Rating Promedio por Género',
        xaxis_title='Género',
        yaxis_title='Cantidad por Género',
        xaxis=dict(
        tickangle=-45 # inclina los nombres de generos en las barras a 45gd
        ),
        yaxis2=dict(
        title='Rating Promedio',
        overlaying='y',
        side='right'),
    
        template='plotly_dark'
    )

        # Mostral el grafico.
    fig_comb.show()
        

pregunta_2()
           
        

El análisis muestra que drama y comedia son géneros con bastante producción y con uno de los mejores ratings.
Sin embargo, otros géneros como musicales, guerra y ciencia ficción no son muy producidos pero son preferidos por el público
debido a sus altos ratings promedio. Esto sugiere que estos géneros podrían ofrecer una buena oportunidad de inversión para 
mejorar la oferta de contenido y atraer a una audiencia que busca calidad en lugar de cantidad.

----------------------------------------------------------------------------------------------------------------------------------

Pregunta 3: ¿Cuáles son los géneros que proporcionan mayor ROI?

In [9]:
def pregunta_3():
    global df_title_basics, df_title_ratings, df_tmdb
    
    
    # separa los generos y crea la columna genero con los generos por separados.
    df_tmdb_genre = df_tmdb.assign(genre=df_tmdb['genres'].str.split(',').explode('genre'))
    
    
    # Eliminar espacios en blanco al inicio y al final de cada genero con strip.
    df_tmdb_genre['genre'] = df_tmdb_genre['genre'].str.strip()
    
    ''' filtral los generos no deseados.
        unwanted_genres es la lista de generos no deseados. 
    '''
    # lista de generos no deseados.
    unwanted_genres = ['Short', 'Adult', 'News', 'Game-Show', 'Adult', 'Talk-Show', 'Western', 'Film-Noir','Thriller', 'TV Movie']
    df_tmdb_genre = df_tmdb_genre[~df_tmdb_genre['genre'].isin(unwanted_genres)]
    
    # crea el df df_genre_roi con los generos agrupados y la media arimetica del roi por genero y lo muestra decendiendo.
    df_tmdb_genre_roi = df_tmdb_genre.groupby('genre')['roi'].mean().sort_values(ascending=False).reset_index()
    df_tmdb_genre_roi.columns = ['Genero', 'ROI Promedio']

    ''' creo un df con los 10 generos con mayor roi.
        redondeo a 3 decimales antes del grafico para que tengo enfecto en el grafico.
    '''
    top_10_roi = round(df_tmdb_genre_roi.head(10), 3)  
    
    # Paleta de colores para el grafico, se pasa como argumento en color_continuous_scale
    #colors = px.colors.sequential.Plasma[1:5] se  puede usar esta opcion si da error.
    colors = ['#0000FF', '#1E90FF', '#4169E1', '#6495ED', '#7B68EE', '#6A5ACD', '#8A2BE2', '#9370DB', '#8B008B', '#9400D3']
    
    
    fig_rio = px.bar(top_10_roi, x='ROI Promedio', y='Genero',
                    title='Top 10 Generos con Mayor ROI Promedio',
                    labels={'Genero':'Genero','ROI Promedio':'ROI Promedio'},
                    orientation='h',
                    template='plotly_dark',
                    color=top_10_roi['ROI Promedio'],
                    color_continuous_scale=colors)

    #fig_rio.update_traces(marker_line_width=1, marker_line_color='rgba(0,0,0,0)')


    # detayes del Grafico.
    fig_rio.update_layout(
        width=1000,
        xaxis_title='RIO Promedio',
        yaxis_title=None,
        title={
            'text': 'Top 10 Generos con Mayor RIO Promedio',
            'y':0.9,
            'x':0.5,
            'xanchor': 'right',
            'yanchor': 'top',
            'pad': dict(t=-30, b=-50)
            },
        font=dict(size=12),
        margin=dict(l=15, t=60)
    
    )

    fig_rio.show()
    
pregunta_3()


El análisis revela que Adventura, sciencia Ficcion y Animacion son los géneros más rentables en términos de ROI.
Sin embargo, géneros como Accion, Fantasia y Guerra,también proporcionan un alto ROI,
lo que los hace atractivos para futuras inversiones.

---------------------------------------------------------------------------------------------------------------------------------

Pregunta 4: ¿Cuáles son los países con mayor producción de contenido?

In [10]:
from cgitb import text


def pregunta_4():
    
    global df_title_basics, df_title_ratings, df_tmdb
    
    '''
     se crea una copia del dF combierto los multiples datos en listas separado por coma ( , ) con assing y luego split
     luego uso explode para poner cada valor de cada lista en la columna production countries.
     y eliminar espacios en blanco con strip.
    '''
    df_tmdb_explode_country = df_tmdb.assign(production_countries=df_tmdb['production_countries'].str.split(',')).explode('production_countries').copy()
    df_tmdb_explode_country['production_countries'] = df_tmdb_explode_country['production_countries'].str.strip() # eliminar espacios en blanco.


    ''' 
      Agrupa por production_countries (produccion por pais), obtiene la media arimetica de la columna roi de cada agrupacion. 
      obtiene un conteo del total de producciones por pais y lo agrega a una nueva columna Count, la columna roi mostrara
      la media arimetica de de produccion por pis y la columna production_countries mostrara el nombre de del pais 
    '''
    df_tmdb_country_plot = df_tmdb_explode_country.groupby('production_countries').agg({'roi': 'mean', 'production_countries': 'count'})
    df_tmdb_country_plot.rename(columns={'production_countries': 'Count'}, inplace=True)
    
    '''reset el indice para combertir  production country como en columna.
       redondeo a 3 decimales
       ordeno por la columna Count de mayor a menor.
    '''
    df_tmdb_country_plot = df_tmdb_country_plot.reset_index()
    df_tmdb_country_plot['roi'] = round(df_tmdb_country_plot['roi'], 3)
    df_tmdb_country_plot = df_tmdb_country_plot.sort_values('Count', ascending=False)
    
    # Crear columna % para las burbujas del grafico. 
    total_production = df_tmdb_country_plot['Count'].sum()
    df_tmdb_country_plot['Porcentage'] = round((df_tmdb_country_plot['Count'] / total_production) * 100, 2).astype(str) + '%'
   
    
    # Creo top 15 para solo mostrar en el grafico los 15 paises con mayor produccion 
    top_15_count = df_tmdb_country_plot.head(15)
    

    
    # Creamos el grafico con plotly.
    
    fig_country = px.scatter(
        top_15_count,
        x='Count',
        y='roi',
        size='Count',
        size_max=130,
        color='roi',
        hover_name='production_countries',
        text='Porcentage', # Agregar los porcentajes como etiquetas
        title='Producción de Contenido por País y ROI "Promedio"',
        template='plotly_dark',
        color_continuous_scale=px.colors.sequential.Viridis # escala de colores.
    )   
    
    # ajustar el tamano de los percentagejes dentro de las bubujas ( tamano de text=)
    def mapeo_size(count):
        min_font = 5
        max_font = 20
        min_count = top_15_count['Count'].min()
        max_count = top_15_count['Count'].max()
        return np.interp(count, [min_count, max_count], [min_font, max_font])
    
    # Aplicar el mapeo de tamaño de fuente
    font_sizes = [mapeo_size(count) for count in top_15_count['Count']]

    fig_country.update_traces(
        textposition='middle center',
        textfont=dict(size=font_sizes, 
                      color='white',
                      family='Arial Black'),
        texttemplate='%{text}',
        marker=dict(line=dict(width=1, color='black'))
    )
    
    
    fig_country.update_layout(
        width=1000,
        xaxis_title="Cantidad de Produccion",
        yaxis_title='ROI Promedio',
        title={
            'text': 'Producción de Contenido por País y ROI Promedio (15 Mas altos)',
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
    
        font=dict(size=12),
        margin=dict(t=30,r=0, b=0, l=0),
        xaxis=dict(
            tickmode="linear",
            tick0=0,
            dtick=500),
        yaxis=dict(
            tickmode='linear',
            tick0=0,
            dtick=0.5)
    )
          
    fig_country.show()
    
pregunta_4()
    

C:\Users\Chamo\AppData\Local\Temp\ipykernel_20112\781378047.py:1: DeprecationWarning:

'cgitb' is deprecated and slated for removal in Python 3.13



Este grafico representa los 15 países con mayor produccion de contenido en peliculas y series de tv.
El tamaño de la burbuja es proporcional a la cantidad de producciones, por lo que las burbujas más grandes representan los
países con más producciones.



-------------------------------------------------------------------------------------------------------------------------------

¿Cual es tu recomendación para el cliente?

La plataforma de streaming debería enfocarse en mantener una alta producción en géneros populares y bien valorados, 
mientras incrementa la inversión en géneros y mercados menos saturados pero altamente rentables. 
Al optimizar el ROI y diversificar el contenido, la plataforma puede maximizar su atractivo para los espectadores y asegurar 
un crecimiento sostenible y rentable.

Enfoque en Series de Televisión:
La plataforma debería centrarse más en las series de televisión que en las películas, ya que, a pesar de tener una menor
producción, las series muestran un rating más estable y una mejor aceptación por parte del público a lo largo de los años.
Sin embargo, no se debe perder de vista las películas, ya que desde 2020 han mostrado un incremento notable en la aceptación
del público, reflejado en su rating.

Impacto del COVID-19:
Es importante considerar el impacto del COVID-19, que comenzó en 2020. Las restricciones y el confinamiento, junto con la mayor
disponibilidad de tiempo libre y la búsqueda de nuevas formas de entretenimiento, probablemente contribuyeron a este cambio en 
la aceptación de las películas. Es posible que en unos años, los ratings de las películas vuelvan a bajar, mientras que las 
series han mostrado una consistencia en ratings superiores a los 7 puntos durante los años.

Géneros Recomendados:
Para incluir en la plataforma, se recomienda enfocarse en géneros con mayor aceptación del público, tales como:
- Musicales
- Drama
- Comedia
- Guerra
- Misterio

La diversidad es buena, pero es esencial centrarse en los géneros que gozan de mayor aceptación por parte del público. 
Tener producciones con mayores ratings atraerá nuevos usuarios y mantendrá a los ya existentes.

Inversión en Producción:
Los géneros con mayor ROI son:
- Aventura
- Ciencia Ficción
- Animación
- Acción
- Fantasía
- Guerra

Exploración de Nuevos Mercados:
Se debe invertir en países que aún no están completamente explotados, pero que tienen un alto ROI en sus producciones, como:
- India
- Corea del Sur
- China

Enfoque en Mercados Predominantes:
Sin perder de vista los mercados predominantes como:
- Estados Unidos
- Reino Unido
- Francia

Conclusión:
La plataforma de streaming debería centrarse en la producción de series de televisión debido a su rating estable y alta 
aceptación a lo largo del tiempo, sin desatender el incremento reciente en la aceptación de las películas. En cuanto a los 
géneros, la plataforma debería enfocarse en aquellos con mayor aceptación y ROI. Además, explorar nuevos mercados con alto ROI 
y mantener una presencia fuerte en los mercados predominantes garantizará un crecimiento sostenible y rentable.